In [1]:
import pandas as pd
import numpy as np
import os
import xlwt
import re

In [2]:
GO = open("/Users/thesis/Desktop/Google Drive/thesis-lorenz/Work in progress.../Databases/GO/go-basic.obo", 'r')

In [3]:
Gene_entry =[]
all_entries =[]
relations=[]
GO_pattern=re.compile(r'GO:[0-9]*')

In [4]:
for line in GO:
    newentry = False
    x = line.split()
    if x !=[]:
            if x[0] == r'[Term]':
                Gene_entry.append(x[0])
            if x[0] == 'id:':
                Gene_entry.append(x[1])
            elif x[0] == 'name:':
                string=''
                for i in range(1,len(x)):
                    string = string + x[i] + ' '
                Gene_entry.append(string)
            elif x[0] == 'namespace:':
                Gene_entry.append(x[1])
            elif x[0] == 'is_a:':
                for i in range(1,len(x)):
                    if GO_pattern.match(x[i]):
                        relations.append(x[i])
            elif x[0] == 'is_obsolete:':
                Gene_entry.append(['obsolete'])
            else:
                continue
    else:
        if Gene_entry != []:
            if relations != []:
                Gene_entry.append(relations)
            if Gene_entry[0] == r'[Term]':
                all_entries.append((Gene_entry[1],Gene_entry[2:]))
                Gene_entry =[]
                relations=[]

In [5]:
GO_entries = dict(all_entries)

Creating ID-name lookup functions

In [6]:
def Find_GO_name (GO_ID):
    return GO_entries[GO_ID][0]

def Find_GO_ID (GO_name):
    for key in GO_entries.keys():
        if GO_name == GO_entries[key][0]:
            return key

Creating parent-child lookup functions

In [7]:
root=[Find_GO_ID('molecular_function '), Find_GO_ID('cellular_component '), Find_GO_ID('biological_process ')]

In [8]:
Map_list = []

for key in GO_entries.keys():
    if key not in root:
        Map_list.append((key,GO_entries[key][2]))
 

In [9]:
GO_Parents = dict(Map_list)
for key in root:
    GO_Parents[key]=[key]

def Find_Parent (GO_ID, levels) => Input is GO_ID en hoeveel 'niveaus' je hoger wilt gaan kijken; output zou dan een lijst moeten zijn met alle termen die 'levels' aantal niveaus hoger liggen
Modification invoeren => MAppen naar termen met afstand x van de root

comment : idealiter gezien wil je het pad tussenchild en parent ook zien maar dat heb ik nog niet gevonden xD Ik vermoed da je deze fct wel kan gebruiken daarvoor maar eerst de deze maken :p

In [10]:
def Find_Parent(GO_ID, levels):
    if GO_Parents[GO_ID] == ['obsolete']:
        return 'This GO ID is obsolete'
    Child_list = []
    Parent_list = [GO_ID]
    for x in range(0,levels):
        Child_list = Parent_list
        Parent_list =[]
        for y in range(0,len(Child_list)):
            Child = Child_list[y]
            Parents = GO_Parents[Child]
            for z in range(0,len(Parents)):
                Parent_list.append(Parents[z])
            #print(Child_list)
            #print (Parent_list)
    return Parent_list
        

Find_Parent('GO:0000001',3) => Geen probleem met lijsten => Parent_list gets appended but child list stays unchanged

In [11]:
def Find_Child(GO_ID,levels):
    if GO_Parents[GO_ID] == ['obsolete']:
        return 'This GO ID is obsolete'
    Child_list = [GO_ID]
    Parent_list = []
    specific_child_list=[]
    for x in range(0,levels):
        Parent_list = Child_list
        Child_list =[]
        for Parent in Parent_list:
            for key in GO_Parents.keys():
                if Parent in GO_Parents[key]:
                    specific_child_list.append(key)
            if specific_child_list != []:
                for term in specific_child_list:
                    Child_list.append(term)
            else:
                Child_list.append(Parent)
                
    return Child_list

In [12]:
def Assign_depth():
    for GO_ID in GO_entries.keys():
        if GO_Parents[GO_ID] == ['obsolete']:
            continue
        root_term = False
        depth =0
        inp=[GO_ID]
        outp=[]
        
        while not root_term:
            depth = depth +1
            for term in inp:
                for parent in Find_Parent(term,1):
                    outp.append(parent)
            
                for word in root:
                    if word in outp:
                        root_term = True
            inp = list(pd.unique(outp[:]))
            outp=[]
            
        GO_entries[GO_ID].append(depth)
    
    
    

In [13]:
Assign_depth()

In [14]:
for key in root:
    GO_entries[key]=GO_entries[key][:2]
    GO_entries[key].append(0)

In [16]:
def Remap (GO_ID, goal_depth):
    
#Tackling special situations
    if goal_depth == 0:
        return Find_GO_ID(GO_entries[GO_ID][2])
    
    if GO_Parents[GO_ID] == ['obsolete']:
        return ('This term is obsolete')
    
    if goal_depth < 0:
        return ("goal_depth can't be a negative integer")
    
#Determining max depth    
    depth_list=[]
    for key in GO_entries.keys():
        if key in root:
            depth_list.append(0)
        else:
            if GO_Parents[key] != ['obsolete']:
                depth_list.append(GO_entries[key][3])
    max_depth = max(pd.unique(depth_list))
    if goal_depth > max_depth:
        return ('max depth in the GO DAG is '+ str(max_depth))
    
#Determining depth of given term    
    if GO_ID in root:
        init_depth=0
    else:
        init_depth = GO_entries[GO_ID][3]
#trivial anwser
    if goal_depth == init_depth:
        return GO_ID
#diff < 0 means goal depth is more specific thus we need to find all child terms at goal depth
#diff>0 means goal depth is more general thus we need to find parents at goal depth
    else:
        diff = init_depth - goal_depth
        if diff < 0: #MAG NIET!!!! Voer check in met waarschuwing
            outlist= []
            for i in range(0,(abs(diff-init_depth))):
                test= Find_Child(GO_ID,i)
                for annotation in test:
                    if GO_entries[annotation][3] == goal_depth:
                        outlist.append(annotation)
            return list(pd.unique(outlist))
        elif diff>0:
            out_list=[]
            for i in range(0,init_depth):
                to_test = Find_Parent(GO_ID,i)
                for term in to_test:
                    if GO_entries[term][3] == goal_depth:
                        out_list.append(term)
            return list(pd.unique(out_list))
                    
                        
                
        
            

In [25]:
Remap('GO:0004012',5)

['GO:0090554', 'GO:0090555', 'GO:0090556', 'GO:0099038']

In [21]:
GO.close()